In [1]:
import cv2
import numpy as np
from math import pi
import math
from scipy.ndimage import filters
from numpy import floor
from math import atan
import matplotlib.pyplot as plt
import os

def show_image(img):
    cv2.imshow('image',img)
    cv2.waitKey(0)
    cv2.destroyAllWindows()

# Weber local descriptor

In [2]:
# Magic numbers
M = 4
T = 12
S = 4

eps = 0.000001

def ratio(x):
    # x[4] - центральный элемент
    value = 0.0
    if x[4] == 0:
        value = pi / 2.0
    else:
        value = atan(float(x.sum() - 9.0 * x[4]) / x[4])
    return value

def intense_fraction(x):
    value = 0.0
    if x[5] == x[1]:
        value = 2.0 * pi
    else:
        value = 2.0 * atan(float(x[7] - x[3]) / (x[5] - x[1])) + pi
    return value

def get_features(image):
    image = [[float(j) for j in i] for i in image]
    # Calculating differential excitation
    image_de = filters.generic_filter(image, ratio, (3, 3))
    # Calculating gradient orientation
    image_teta = filters.generic_filter(image, intense_fraction, (3, 3))
    # Calculating dominant orientation
    phi = 2.0 * pi * (floor((image_teta * T) / (2.0 * pi) + 1.0 / 2.0) % T) / T
    # Each phi[j] is from T values: {phi_0, phi_1, ....phi_(T-1)}
    H_t = []
    for t in range(T):
        phi_t = 2.0 * pi * t / T 
        # Indicies x and y coordinates   
        indicies = np.where(abs(phi - phi_t) < eps)
        # We wil build histogram on diff excitiation
        values = image_de[indicies[0], indicies[1]]
        # In H_t (array_of_values, array_of_coords)
        hist, bins = np.histogram(values, bins=M * S)
        H_t.append(hist)
        # width = 0.7 * (bins[1] - bins[0])
        # center = (bins[:-1] + bins[1:]) / 2
        # plt.bar(center, hist, align='center', width=width)
        # plt.show()
    
    # Divide each of t histograms to m parts
    length = len(H_t[0])
    # print "length = " + str(length)
    # print "first hist:"
    # print H_t[0]
    # -----> t
    # |
    # |
    # |
    # |
    # v m
    H_tm = []
    for t in range(T):
        hist_part = []
        for m in range(M):
            hist_part.append(H_t[t][m * length / M: (m + 1) * length / M])
        H_tm.append(hist_part)

    # H_tm = [hist_part_1, ..., hist_part_T],
    # where hist_part_t = [hist_1, .., hist_M]
    H_m = []
    for m in range(M):
        hist_m = []
        for t in range(T):
            hist_m += [value for value in H_tm[t][m]]
        H_m.append(hist_m)

    H = []
    for i in range(M):
        H += H_m[i]
        
    # N = len(H)
    #x = range(N)
    #width = 1/1.5
    #plt.bar(x, H, width, color="blue")
    #plt.show()
    return H

def WLD(image, num_of_blocks=9):
    height = len(image)
    width = len(image[0])
    features = []
    if num_of_blocks == 4:
        features += get_features(image[0:height/2, 0:width/2])
        features += get_features(image[0:height/2, width/2:width])
        features += get_features(image[height/2:height, 0:width/2])
        features += get_features(image[height/2:height, width/2:width])
        return features
    elif num_of_blocks == 9:
        x_1_3 = width * 1 / 3
        x_2_3 = width * 2 / 3
        y_1_3 = height * 1 / 3
        y_2_3 = height * 2 / 3
        features += get_features(image[0:y_1_3,0:x_1_3])
        features += get_features(image[y_1_3:y_2_3,0:x_1_3])
        features += get_features(image[y_2_3:height,0:x_1_3])
        features += get_features(image[0:y_1_3,x_1_3:x_2_3])
        features += get_features(image[y_1_3:y_2_3,x_1_3:x_2_3])
        features += get_features(image[y_2_3:height,x_1_3:x_2_3])
        features += get_features(image[0:y_1_3,x_2_3:width])
        features += get_features(image[y_1_3:y_2_3,x_2_3:width])
        features += get_features(image[y_2_3:height,x_2_3:width])   
        return features
    elif num_of_blocks == 1:
        features += get_features(image)
        return features
    else:
        print str(num_of_blocks) + " : Number of blocks not supported"

# Обучаемся

In [3]:
# STUB: пока что только фронтальный лица
X_train = []
labels_train = []

face_cascade = cv2.CascadeClassifier('haarcascade_frontalface_default.xml')

female_train_path = os.path.abspath("train_dataset/female")
male_train_path = os.path.abspath("train_dataset/male")

# FEMALE
print "Female pictures:"
for filename in os.listdir(female_train_path):
    print filename
    for picture in os.listdir(os.path.join(female_train_path, filename)):
         if picture.endswith(".jpg"): 
            picture_path = os.path.abspath(os.path.join(os.path.join(female_train_path, filename), picture))
            img = cv2.imread(picture_path)
            gray_img = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
            faces = face_cascade.detectMultiScale(gray_img, 1.3, 5)
            for (x, y, w, h) in faces:
                x_stop = x + w
                y_stop = y + h
                # rect = cv2.rectangle(img,(x,y),(x_stop,y_stop),(0,255,0),2)
                X_train.append(WLD(gray_img[y:y_stop, x:x_stop]))
                labels_train.append(0)
                # Здесь одно лицо на каждом образце
                break

Female pictures:
9336923
9338535
9540814
9540849
amflem
anpage
aolcer
apapou
asamma
asewil
astefa
boylee
drbost
ekavaz
elduns
gghazv
gotone
hsgrim
isbald
jbierl
jross
kaknig
klclar
ksunth
labenm
lcelli
ldgodd
lfso
mbutle
mclarkd
mkosto
mkotza
phughe
rhosan
sbains
shpill
slbirc
smfarrf
vanta
vstros
wylsow
yfhsie


In [4]:
# MALE
print "Male pictures:"
for filename in os.listdir(male_train_path):
    print filename
    for picture in os.listdir(os.path.join(male_train_path, filename)):
         if picture.endswith(".jpg"): 
            picture_path = os.path.abspath(os.path.join(os.path.join(male_train_path, filename), picture))
            img = cv2.imread(picture_path)
            gray_img = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
            faces = face_cascade.detectMultiScale(gray_img, 1.3, 5)
            for (x, y, w, h) in faces:
                x_stop = x + w
                y_stop = y + h
                # rect = cv2.rectangle(img,(x,y),(x_stop,y_stop),(0,255,0),2)
                X_train.append(WLD(gray_img[y:y_stop, x:x_stop]))
                labels_train.append(1)
                # Здесь одно лицо на каждом образце
                break

Male pictures:
9326871
9332898
9338446
9338454
9338462
9338489
9338497
9338519
9338527
9338543
9414649
9416994
admars
ahodki
ajflem
ajones
ajsega
akatsi
ambarw
anonym
anonym1
anonym2
asheal
bplyce
cchris
ccjame
cferdo
cgboyc
cjcarr
cjdenn
cjsake
cmkirk
csanch
cshubb
cwang
cwchoi
dagran
dakram
dcbowe
dioann
djbirc
djhugh
djmart
dmwest
doraj
fordj
gdhatc
ggeorg
ggrego
gjhero
gjnorm
gmwate
gpapaz
gpsmit
gsreas
hartb
hensm
ieorf
irdrew
jabins
jagrif
jcarte
jdbenm
jgloma
jlemon
jmedin
jrtobi
kaatki
kdjone
khchan
khughe
kjwith
lejnno
lyond
maasht
macci
martin
mberdo
mdpove
mefait
mhwill
miaduc
michael
mjhans
moors
mpetti
muthay
nahaig
namull
ndbank
ndhagu
nhrams
njmoor
npbour
npmitc
nrclar
nrrbar
nwilli
obeidn
ohpark
pacole
pmives
pshurr
pspliu
ptnich
rarobi
rgharr
rgspru
rjlabr
rlocke
rmcoll
rmpugh
rnpwil
robin
rrowle
rsanti
saduah
saedwa
sandm
sidick
sjbeck
skumar
smrobb
spacl
spletc
svkriz
swewin
swsmit
tony
voudcx
vpsavo
whussa
wjalbe


In [5]:
female_grimace_path = "grimace/female"
# FEMALE
print "Female grimaces:"
for filename in os.listdir(female_grimace_path):
    print filename
    for picture in os.listdir(os.path.join(female_grimace_path, filename)):
         if picture.endswith(".jpg"): 
            picture_path = os.path.abspath(os.path.join(os.path.join(female_grimace_path, filename), picture))
            img = cv2.imread(picture_path)
            gray_img = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
            faces = face_cascade.detectMultiScale(gray_img, 1.3, 5)
            for (x, y, w, h) in faces:
                x_stop = x + w
                y_stop = y + h
                # rect = cv2.rectangle(img,(x,y),(x_stop,y_stop),(0,255,0),2)
                X_train.append(WLD(gray_img[y:y_stop, x:x_stop]))
                labels_train.append(0)
                # Здесь одно лицо на каждом образце
                break

Female grimaces:
chr
sar


In [6]:
# MALE
male_grimace_path = "grimace/male"
print "Male grimaces:"
for filename in os.listdir(male_grimace_path):
    print filename
    for picture in os.listdir(os.path.join(male_grimace_path, filename)):
         if picture.endswith(".jpg"): 
            picture_path = os.path.abspath(os.path.join(os.path.join(male_grimace_path, filename), picture))
            img = cv2.imread(picture_path)
            gray_img = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
            faces = face_cascade.detectMultiScale(gray_img, 1.3, 5)
            for (x, y, w, h) in faces:
                x_stop = x + w
                y_stop = y + h
                # rect = cv2.rectangle(img,(x,y),(x_stop,y_stop),(0,255,0),2)
                X_train.append(WLD(gray_img[y:y_stop, x:x_stop]))
                labels_train.append(1)
                # Здесь одно лицо на каждом образце
                break

Male grimaces:
and
ant
dah
dav
den
glen
ian
jer
john
lib
mike
pat
ste
stu
tom
will


In [7]:
print "And data is:"
print len(X_train)
print len(labels_train)

And data is:
3717
3717


# На тестовой

In [8]:
# Парсим тестовую базу
import csv
files_to_X = dict()
with open('test_dataset//markup.csv', 'rb') as csvfile:
    data = csv.reader(csvfile, delimiter=',')
    for raw in data:
        file_name = raw[0].strip() + ".tiff"
        if file_name not in files_to_X:
            files_to_X[file_name] = list()
        files_to_X[file_name].append([raw[1].strip(), raw[2].strip(), raw[3].strip(), raw[4].strip(), raw[5].strip()])     

In [9]:
# Проверяем на ней
X = []
labels = []
for file, faces in files_to_X.iteritems():
    print file
    img = cv2.imread("test_dataset\\" + file)
    gray_img = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
    for face in faces:
        x = int(face[0])
        y = int(face[1])
        w = int(face[2]) - x
        h = int(face[3]) - y
        x_stop = x + w
        y_stop = y + h
        if face[4] == 'f':
            labels.append(0)
        else:
            labels.append(1)
        X.append(WLD(gray_img[y:y_stop, x:x_stop]))

0000009.tiff
0000030.tiff
0000033.tiff
0000013.tiff
0000049.tiff
0000048.tiff
0000050.tiff
0000029.tiff
0000031.tiff
0000027.tiff
0000010.tiff
0000053.tiff
0000028.tiff
0000046.tiff
0000012.tiff
0000011.tiff
0000032.tiff
0000044.tiff
0000023.tiff
0000047.tiff
0000051.tiff


In [10]:
from sklearn.neighbors import KNeighborsClassifier
nbrs = KNeighborsClassifier(n_neighbors=4, metric='canberra')
nbrs.fit(X_train, labels_train)
predicted = nbrs.predict(X)
target_names = ['female', 'male']
from sklearn.metrics import classification_report
print classification_report(labels, predicted, target_names=target_names) 

             precision    recall  f1-score   support

     female       0.70      0.38      0.49       110
       male       0.48      0.78      0.60        82

avg / total       0.61      0.55      0.54       192



# Print in file

In [12]:
counter = 0
for file, faces in files_to_X.iteritems():
    print file
    img = cv2.imread("test_dataset\\" + file)
    gray_img = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
    for face in faces:
        x = int(face[0])
        y = int(face[1])
        x_stop = int(face[2])
        y_stop = int(face[3])
        sex = ''
        if predicted[counter] == 1:
            sex = 'm'
        else:
            sex = 'f'
        with open('output.csv', 'ab') as csvfile:
            writer = csv.writer(csvfile, delimiter=',', quoting=csv.QUOTE_MINIMAL)
            row = [file[:-5], x, y, x_stop, y_stop, sex]
            writer.writerow(row)
        counter += 1


0000009.tiff
0000030.tiff
0000033.tiff
0000013.tiff
0000049.tiff
0000048.tiff
0000050.tiff
0000029.tiff
0000031.tiff
0000027.tiff
0000010.tiff
0000053.tiff
0000028.tiff
0000046.tiff
0000012.tiff
0000011.tiff
0000032.tiff
0000044.tiff
0000023.tiff
0000047.tiff
0000051.tiff


# WIKI dataset

In [ ]:
import scipy.io
mat = scipy.io.loadmat('wiki/wiki.mat')

In [ ]:
wiki_format = {
    'dob': 0,
    'photo_taken': 1,
    'full_path': 2,
    'gender': 3,
    'name': 4,
    'face_location': 5,
    'face_score': 6,
    'second_face_score': 7
}

genders = []
face_location = []
file_path = []
face_score = []
second_face_score = []

wiki_dataset = mat['wiki']
for i in range(1):
    for j in range(1):
        for k in range(8):
            if wiki_format['gender'] == k:
                genders = wiki_dataset[i][j][k][0]
            elif wiki_format['full_path'] == k:
                file_path = wiki_dataset[i][j][k][0]
            elif wiki_format['face_location'] == k:
                face_location = wiki_dataset[i][j][k][0]
            elif wiki_format['face_score'] == k:
                face_score = wiki_dataset[i][j][k][0]
            elif wiki_format['second_face_score'] == k:
                second_face_score = wiki_dataset[i][j][k][0]

In [ ]:
import math

def image_fits(gender, path, score1, score2, face_coords):
    # print gender, path, score1, score2, face_coords
    fits = True
    fits = (fits) and (path != "")
    fits = (fits) and (not math.isnan(gender)) and ((int(gender) == 0) or (int(gender) == 1))
    fits = (fits) and (len(face_coords) == 4)
    fits = (fits) and (not math.isinf(score1))
    fits = (fits) and (math.isnan(score2))
    return fits

N = len(file_path)
print N
X_wiki = []
labels_wiki = []
for i in range(N):
    if i == 10000:
        break
    if image_fits(genders[i], file_path[i], face_score[i], second_face_score[i], face_location[i][0]):
        path = os.path.abspath(os.path.join("wiki", file_path[i][0]))
        img = cv2.imread(path, 0)
        if img == None:
            continue
        # gray_img = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
        x = int(face_location[i][0][0])
        y = int(face_location[i][0][1])
        x_stop = int(face_location[i][0][2])
        y_stop = int(face_location[i][0][3])
        # rect = cv2.rectangle(img,(x,y),(x_stop,y_stop),(0,255,0),2)
        # show_image(rect)
        q = WLD(img[y:y_stop, x:x_stop])
        if np.isnan(q).any():
            continue
        labels_wiki.append(int(genders[i]))
        X_wiki.append(q)

In [ ]:
print len(X_wiki)
print len(labels_wiki)

# Проверяем с WIKI

In [ ]:
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import classification_report
from sklearn.ensemble import RandomForestClassifier
model_wiki = RandomForestClassifier(n_estimators=100)
model_wiki.fit(X_wiki, labels_wiki)
predicted = model_wiki.predict(X)
target_names = ['female', 'male']
print classification_report(labels, predicted, target_names=target_names) 

In [ ]:
nbrs = KNeighborsClassifier(n_neighbors=5, metric='canberra')
nbrs.fit(X_wiki, labels_wiki)
predicted = nbrs.predict(X)
target_names = ['female', 'male']
from sklearn.metrics import classification_report
print classification_report(labels, predicted, target_names=target_names) 